In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

In [2]:
# Load documents
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=1200)
tool= WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
tool.name

'wikipedia'

## Data Ingestion

In [1]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from src.constants import traning_pipline
url=traning_pipline.DATA_INGESTION_DATA_URL

In [4]:
#url='https://en.wikipedia.org/wiki/DeepSeek'
loder=WebBaseLoader(url)
docs=loder.load()

In [5]:
all_text=''
filename=r'D:\assamement-project\Wikipedia-RAG-Based-Application\data\data.txt'
if docs:
   all_text += f"=== {url} ===\n{docs[0].page_content}\n\n"

In [7]:
import os
dir_path = os.path.dirname(filename)
os.makedirs(dir_path, exist_ok=True)

In [8]:
with open(filename, "w", encoding="utf-8") as f:
   f.write(all_text)

In [16]:
len(all_text)

43797